# ERA5-Land Hourly Data

ERA5-Land is a high-resolution reanalysis dataset that provides a consistent and detailed view of land variables over several decades, combining model data with atmospheric forcing from ERA5 to ensure accuracy. By correcting input variables for altitude differences and leveraging indirect observational influences, it offers enhanced precision for land surface applications like flood and drought forecasting. Despite some inherent uncertainties, ERA5-Land's extensive temporal and spatial resolution makes it a valuable resource for decision-making and environmental analysis.

* [ERA5-Land Hourly Data](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land?tab=overview')
* [Documentation](https://confluence.ecmwf.int/display/CKB/ERA5-Land%3A+data+documentation)

## Import Necessary Libraries

In [1]:
import os
import ipywidgets as widgets
from datetime import datetime

from cds_client_util import initialize_cds_client

## Manage Working Directories

In [2]:
# The directories (if doesn't exist) will be created in the Data folder
download_folder = r".\data\era5-land-hourly-data\download"
working_folder = r".\data\era5-land-hourly-data\working"
output_folder = r".\data\era5-land-hourly-data\output"

if not os.path.exists(working_folder):
    os.makedirs(working_folder)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

## CDS API key and authentication

In [3]:
# Use the function to initialize the client
try:
    client = initialize_cds_client("../secrets/cds_api")
    print("CDS API Client initialized successfully!")
except (FileNotFoundError, ValueError) as e:
    print(f"Error: {e}")

2024-11-25 15:14:03,971 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.


2024-11-25 15:14:03,978 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


2024-11-25 15:14:03,980 INFO [2024-09-16T00:00:00] Remember that you need to have an ECMWF account to use the new CDS. **Your old CDS credentials will not work in new CDS!**


2024-11-25 15:14:03,982 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.


CDS API Client initialized successfully!


In [4]:
# Alternative: Enter your CDS API key directly
# ------------------------------------------------------
# If you have not created the ".cdsapirc" file or prefer not to use it, 
# you can manually input your CDS API key in this notebook cell.
# This approach is helpful for quick testing or one-off uses but is not recommended 
# for long-term or automated workflows as it requires manual input and may expose your key.
# ------------------------------------------------------

# import getpass

# api_key = getpass.getpass("Enter your CDS API Key:")
# api_url = "https://cds.climate.copernicus.eu/api"
# client = cdsapi.Client(url=api_url, key=api_key)

## Create list of variables

The list of variables are available at [ERA5-Land Hourly](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land?tab=download)

*Please check and modify the predefined variable name if any issue arises with variable name.* 

In [5]:
# Variable Groups
var_group_temperature = [
    "2m_dewpoint_temperature",
    "2m_temperature",
    "skin_temperature",
    "soil_temperature_level_1",
    "soil_temperature_level_2",
    "soil_temperature_level_3",
    "soil_temperature_level_4"]

var_group_lake = [
    "lake_bottom_temperature",
    "lake_ice_depth",
    "lake_ice_temperature",
    "lake_mix_layer_depth",
    "lake_mix_layer_temperature",
    "lake_shape_factor",
    "lake_total_layer_temperature"]

var_group_snow = [
    "snow_albedo",
    "snow_cover",
    "snow_density",
    "snow_depth",
    "snow_depth_water_equivalent",
    "snowfall",
    "snowmelt",
    "temperature_of_snow_layer"]

var_group_soil_water = [
    "skin_reservoir_content",
    "volumetric_soil_water_layer_1",
    "volumetric_soil_water_layer_2",
    "volumetric_soil_water_layer_3",
    "volumetric_soil_water_layer_4"]

var_group_radiation_and_heat = [
    "forecast_albedo",
    "surface_latent_heat_flux",
    "surface_net_solar_radiation",
    "surface_net_thermal_radiation",
    "surface_sensible_heat_flux",
    "surface_solar_radiation_downwards",
    "surface_thermal_radiation_downwards"]

var_group_evaporation_and_runoff = [
    "evaporation_from_bare_soil",
    "evaporation_from_open_water_surfaces_excluding_oceans",
    "evaporation_from_the_top_of_canopy",
    "evaporation_from_vegetation_transpiration",
    "potential_evaporation",
    "runoff",
    "snow_evaporation",
    "sub_surface_runoff",
    "surface_runoff",
    "total_evaporation"]

var_group_wind_pressure_and_precipitation = [
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "surface_pressure",
    "total_precipitation"]

var_group_vegetation = [
    "leaf_area_index_high_vegetation",
    "leaf_area_index_low_vegetation"]

# List of variable group
var_group_list = ['var_group_temperature',
                  'var_group_lake',
                  'var_group_soil_water',
                  'var_group_radiation_and_heat',
                  'var_group_evaporation_and_runoff',
                  'var_group_wind_pressure_and_precipitation',
                  'var_group_vegetation']

## Define request parameters for download

In [6]:
selected_variable_group = widgets.Dropdown(
    options=var_group_list,
    value=var_group_list[0],
    description="Select a variable group",
    style=dict(description_width='initial'),
    layout=widgets.Layout(width='50%'),
)

selected_variable_group

Dropdown(description='Select a variable group', layout=Layout(width='50%'), options=('var_group_temperature', …

In [7]:
current_variable_group = globals().get(selected_variable_group.value)

selected_variable = widgets.Dropdown(
    options=current_variable_group,
    value=current_variable_group[0],
    description="Select the variable of interest",
    style=dict(description_width='initial'),
    layout=widgets.Layout(width='50%'),
)

selected_variable

Dropdown(description='Select the variable of interest', layout=Layout(width='50%'), options=('2m_dewpoint_temp…

### Define Bounding Box Extents (Bbox) for the Dataset

- Modify the code below to manually set the desired bounding box values, or
- Use the interactive bounding box tool by visiting this [Bbox-extractor](https://str-ucture.github.io/bbox-extractor/), which allows you to easily generate and visualize the bounding box coordinates for your dataset.

In [8]:
# Define the bounding box extents in CRS: WGS84 in this format [lat_max, lon_min, lat_min, lon_max]
# Recommended precision is 0.1 because Horizontal resolution: 0.1° x 0.1°; Native resolution is 9 km.

bbox_wgs84_deutschland = [56.0, 5.8, 47.2, 15.0] # Bouding box for Germany
bbox_wgs84_konstanz = [47.8, 9, 47.6, 9.3] # Bounding box for Konstanz, Germany

In [9]:
selected_year = widgets.Dropdown(
    options=[str(year) for year in range(1950, datetime.now().year + 1)],
    value=str(datetime.now().year),
    description="Select the year for downloading data:",
    disabled=False,
    style=dict(description_width='initial'),
    layout=widgets.Layout(width='50%'),
)

selected_year

Dropdown(description='Select the year for downloading data:', index=74, layout=Layout(width='50%'), options=('…

## Define request parameters and download the dataset

In [10]:
# This request downloads 1 year data: 12 months (365 days, 366 days for leap years) and 24 hour data for each day.
# The total length of the downloaded dataset is 365*24=8760 and 366*24=8784 for leap year.
# Exception: For year 1950, length of dataset = 8759

dataset = "reanalysis-era5-land"
dataset_filename = f"{dataset}_{selected_variable.value}_{selected_year.value}.nc"
dataset_filepath = os.path.join(download_folder,dataset_filename)

request = {
    "variable": selected_variable.value,
    "year": selected_year.value,
    "month": [str(month) for month in range(13)],
    "day": [str(day) for day in range(32)],
    "time": [f"{hour:02d}:00" for hour in range(24)],
    "data_format": "netcdf",
    "download_format": "unarchived",
    "area": bbox_wgs84_konstanz
}

# Download the dataset with the defined request paremeters
client.retrieve(dataset, request, dataset_filepath)

2024-11-25 15:14:04,471 INFO Request ID is 18f563d2-40ff-45e0-a525-f3c932918efa


2024-11-25 15:14:04,568 INFO status has been updated to accepted


2024-11-25 15:14:07,701 INFO status has been updated to running


---

## Tips on Updating the Request Parameters (`dict`)

Here are some additional details and examples to help you customize the `request` dictionary:

## General Notes:
- **Key Values**: 
  - All values in the dictionary must be in string (`str`) format except for `area`. 
  - The `area` key is a geographic bounding box in `WGS84` coordinate reference system format.
- **Formatting**: Always ensure date and time formats conform to the required structure (e.g., `year` as `"YYYY"`, `time` as `"HH:MM"`).

<hr style="border: none; border-top: 1px solid #ddd; margin: 20px 0;">

### Customizing Parameters:

#### 1. Variable (`variable`):
- You can specify a **single variable** or a **list of variables**.
- **Example**:
  ```python
  "variable": "2m_temperature"
  "variable": ["2m_temperature", "snow_cover", "precipitation"]
  
#### 2. Year (`year`):
- Define year as a **single year** or a **range of years** or a **list of years**.
- **Example**:
  ```python
  "year": "2020"
  "year": ["1950", "1960", "1965"]
  "year": [str(year) for year in range(2018,2023,1)]
> **_NOTE:_**  For Land5-Land hourly data, it is recommended to download data for 1 year at a time.

#### 3. Month (`month`):
- Use string to represent months from "01" to "12"
- **Example**:
  ```python
  "month": "01"
  "month": ["01", "02", "03"]
  "month": [str(month).zfill(2) for month in range(1, 13)]

#### 4. Day (`day`):
- Days should be strings from "01" to "31".
- **Example**:
  ```python
  "day": "01"
  "day": ["01", "02", "03"]
  "day": [str(day).zfill(2) for day in range(1, 32)]

> **_NOTE:_** The logic for handling months with fewer than 31 days is already built-in. Only valid days (dates) for each month will be downloaded, ensuring that the request does not include invalid dates like February 30 or April 31.

#### 5. Time (`time`):
- Define hourly timestamps in HH:MM format.
- **Example**:
  ```python
  "time": "12:00",
  "time": ["00:00", "12:00", "18:00"]
  "time": [f"{hour:02d}:00" for hour in range(24)]

#### 6. Data Format (`data_format`):
- Supported formats: "netcdf" and "grib"

#### 7. Download Format (`download_format`):
- Specify wherher the file should be archieved or unarchieved
- Generally, multiple years are downloaded as archived files (.zip), while a single year is downloaded as unarchived files (.nc, .grib)

#### 8. Area (`area`):
- The areas must be defined as a geographic bounding box in WGS84 coordinate reference system.